# Step 1 : import Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# step 2: Load the dataset

In [ ]:
# Load the dataset CSV file
df = pd.read_csv('/content/zomato.csv')

# Preview the last 5 rows
print(df.tail())


# Step 3: create required features and labels for model training

In [ ]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Extract date features
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Create the target column: Next Day Close Price
df['Next_Close'] = df['Close'].shift(-1)

# Drop the last row from target (as it is a null value)
df = df.dropna()

print(df[['Date', 'Open', 'High', 'Low', 'Close', 'Next_Close', 'DayOfWeek', 'Month', 'Year', 'Volume']].tail())

# Step 4: Define input and output datasets

In [ ]:
# Define input features
X = df[['Open', 'High', 'Low', 'Close', 'Volume', 'DayOfWeek', 'Month', 'Year']]

# Define target output
y = df['Next_Close']

# Step 5: Split dataset into training and test

In [ ]:
split_index = int(len(df) * 0.8)

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]


# Step 6 : Use Standard Scalar

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# step 7: Deinfe Model and fit

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# step 8: Predict output from trained model and calculate Mean Absolute Error, R2 score

In [ ]:
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)

print(f'MAE: {mae}')

In [ ]:
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, y_pred)
print(f'R2 Score: {r2_score}')

# Step 9: Save the model

In [ ]:
import joblib
# save model
joblib.dump(model, "Model.pkl")
joblib.dump(scaler, "Scaler.pkl")
print("model saved")

# Step 10: load the model

In [ ]:
Model = joblib.load("Model.pkl")
scaler = joblib.load("Scaler.pkl")
print("model lodeded.")

# Step 11: Create function to predict the Price

In [ ]:
import numpy as np
def predict_stock_price(open, high, low, close, volume, date_str):
  # convert string to datetime
  date = pd.to_datetime(date_str)
  # extract date features
  day_of_week = date.dayofweek
  month = date.month
  year = date.year

  # create a numpy array
  input_data = np.array([[open, high, low, close, volume, day_of_week, month, year]])
  input_scaled = scaler.transform(input_data)
  result = Model.predict(input_scaled)
  return result[0]


# Step 12: Use Model

In [ ]:
# take input from user:
open = float(input("Enter the open price: "))
if open < 0:
  print("invalid input")
  exit()
high = float(input("enter high: "))
if high < 0:
  print("invalid input")
  exit()
low = float(input("enter low: "))
if low < 0:
  print("invalid input")
  exit()
close = float(input("enter close: "))
if close < 0:
  print("invalid input")
  exit()
volume = float(input("enter volume: "))
if volume < 0:
  print("invalid input")
  exit()
date = input("enter date [eg: year-month-date]: ")
if date == "":
  print("invalid input")
  exit()
# call the function
StockPrice = predict_stock_price(open, high, low, close, volume, date)
print(f"stock pricee : Rs {StockPrice:.2f}")